In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6421,2020-12-25T17:00:00,ITA,19,Sicilia,38.115697,13.362357,995,174,1169,32063,...,2256,NaN,NaN,88597,1179159,788040.0,NaN,15.0,NaN,NaN
6422,2020-12-25T17:00:00,ITA,9,Toscana,43.769231,11.255889,843,166,1009,10260,...,3558,NaN,NaN,118155,1841106,1047247.0,NaN,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6423,2020-12-25T17:00:00,ITA,10,Umbria,43.106758,12.388247,240,36,276,3270,...,597,NaN,NaN,27977,490891,248275.0,NaN,3.0,NaN,NaN
6424,2020-12-25T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,84,4,88,340,...,371,NaN,NaN,7130,61109,36873.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6421,2020-12-25T17:00:00,19,Sicilia,995,174,1169,32063,33232,-148,720,53109,2256,NaN,NaN,88597,1179159,788040.0,15.0,NaN,NaN
6422,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,103328,3558,NaN,NaN,118155,1841106,1047247.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6423,2020-12-25T17:00:00,10,Umbria,240,36,276,3270,3546,25,232,23834,597,NaN,NaN,27977,490891,248275.0,3.0,NaN,NaN
6424,2020-12-25T17:00:00,2,Valle d'Aosta,84,4,88,340,428,1,11,6331,371,NaN,NaN,7130,61109,36873.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-25', '2020-12-24')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-25T17:00:00,1,Piemonte,3029,214,3243,28254,31497,-3373,875,154123,7755,NaN,NaN,193375,1648345,972116.0,7.0,NaN,NaN
1,2020-12-25T17:00:00,2,Valle d'Aosta,84,4,88,340,428,1,11,6331,371,NaN,NaN,7130,61109,36873.0,0.0,NaN,NaN
2,2020-12-25T17:00:00,3,Lombardia,3976,522,4498,54212,58710,802,2628,386391,24782,NaN,NaN,469883,4765500,2577061.0,20.0,NaN,NaN
3,2020-12-25T17:00:00,5,Veneto,2582,346,2928,84457,87385,-16637,5010,141454,5953,NaN,NaN,234792,3201904,1187079.0,36.0,NaN,NaN
4,2020-12-25T17:00:00,6,Friuli Venezia Giulia,572,54,626,11458,12084,-18,584,33681,1534,NaN,NaN,47299,904549,334240.0,4.0,NaN,NaN
5,2020-12-25T17:00:00,7,Liguria,663,64,727,4942,5669,-289,445,50460,2842,NaN,NaN,58971,693519,317936.0,3.0,I 145443 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-25T17:00:00,8,Emilia-Romagna,2711,206,2917,56063,58980,1681,2127,96910,7400,NaN,NaN,163290,2496457,1204670.0,16.0,NaN,NaN
7,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,103328,3558,NaN,NaN,118155,1841106,1047247.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-25T17:00:00,10,Umbria,240,36,276,3270,3546,25,232,23834,597,NaN,NaN,27977,490891,248275.0,3.0,NaN,NaN
9,2020-12-25T17:00:00,11,Marche,398,60,458,9109,9567,152,603,28227,1516,NaN,NaN,39310,511362,303386.0,3.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-24T17:00:00,1,Piemonte,3144,213,3357,31513,34870,-1691,1057,149903,7727,NaN,NaN,192500,1642265,969893.0,7.0,NaN,NaN
1,2020-12-24T17:00:00,2,Valle d'Aosta,83,4,87,340,427,-3,35,6321,371,NaN,NaN,7119,61008,36840.0,0.0,NaN,Vi è stato un errore tecnico riguardante terap...
2,2020-12-24T17:00:00,3,Lombardia,4178,521,4699,53209,57908,-2331,2656,384670,24677,NaN,NaN,467255,4737558,2567307.0,14.0,NaN,NaN
3,2020-12-24T17:00:00,5,Veneto,2630,351,2981,101041,104022,696,3837,119901,5859,NaN,NaN,229782,3188128,1183080.0,30.0,NaN,NaN
4,2020-12-24T17:00:00,6,Friuli Venezia Giulia,582,56,638,11464,12102,-173,831,33099,1514,NaN,NaN,46715,898714,332699.0,4.0,NaN,NaN
5,2020-12-24T17:00:00,7,Liguria,687,64,751,5207,5958,-219,328,49737,2831,NaN,NaN,58526,689389,316700.0,4.0,I 143027 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-24T17:00:00,8,Emilia-Romagna,2718,201,2919,54380,57299,-726,1692,96529,7335,NaN,NaN,161163,2485596,1200124.0,8.0,NaN,NaN
7,2020-12-24T17:00:00,9,Toscana,888,161,1049,9920,10969,-388,563,103021,3552,NaN,NaN,117542,1830077,1043441.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-24T17:00:00,10,Umbria,238,38,276,3245,3521,-27,192,23633,591,NaN,NaN,27745,488681,247406.0,4.0,NaN,NaN
9,2020-12-24T17:00:00,11,Marche,409,64,473,8942,9415,-144,415,27788,1504,NaN,NaN,38707,504651,298324.0,6.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-25T17:00:00,1,Piemonte,3029,214,3243,28254,31497,-3373,875,154123,7755,NaN,NaN,193375,1648345,972116.0,7.0,NaN,NaN
1,2020-12-25T17:00:00,2,Valle d'Aosta,84,4,88,340,428,1,11,6331,371,NaN,NaN,7130,61109,36873.0,0.0,NaN,NaN
2,2020-12-25T17:00:00,3,Lombardia,3976,522,4498,54212,58710,802,2628,386391,24782,NaN,NaN,469883,4765500,2577061.0,20.0,NaN,NaN
3,2020-12-25T17:00:00,4,Trentino-Alto Adige,453,68,521,11820,12341,1488,336,35617,1603,0.0,0.0,49561,794650,311080.0,4.0,0,NaN
4,2020-12-25T17:00:00,5,Veneto,2582,346,2928,84457,87385,-16637,5010,141454,5953,NaN,NaN,234792,3201904,1187079.0,36.0,NaN,NaN
5,2020-12-25T17:00:00,6,Friuli Venezia Giulia,572,54,626,11458,12084,-18,584,33681,1534,NaN,NaN,47299,904549,334240.0,4.0,NaN,NaN
6,2020-12-25T17:00:00,7,Liguria,663,64,727,4942,5669,-289,445,50460,2842,NaN,NaN,58971,693519,317936.0,3.0,I 145443 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-25T17:00:00,8,Emilia-Romagna,2711,206,2917,56063,58980,1681,2127,96910,7400,NaN,NaN,163290,2496457,1204670.0,16.0,NaN,NaN
8,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,103328,3558,NaN,NaN,118155,1841106,1047247.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-25T17:00:00,10,Umbria,240,36,276,3270,3546,25,232,23834,597,NaN,NaN,27977,490891,248275.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-24T17:00:00,1,Piemonte,3144,213,3357,31513,34870,-1691,...,149903,7727,NaN,NaN,192500,1642265,969893.0,7.0,NaN,NaN
1,1,2020-12-24T17:00:00,2,Valle d'Aosta,83,4,87,340,427,-3,...,6321,371,NaN,NaN,7119,61008,36840.0,0.0,NaN,Vi è stato un errore tecnico riguardante terap...
2,2,2020-12-24T17:00:00,3,Lombardia,4178,521,4699,53209,57908,-2331,...,384670,24677,NaN,NaN,467255,4737558,2567307.0,14.0,NaN,NaN
3,19,2020-12-24T17:00:00,4,Trentino-Alto Adige,483,66,549,10304,10853,-27,...,36781,1591,0.0,0.0,49225,789649,309332.0,4.0,0,NaN
4,3,2020-12-24T17:00:00,5,Veneto,2630,351,2981,101041,104022,696,...,119901,5859,NaN,NaN,229782,3188128,1183080.0,30.0,NaN,NaN
5,4,2020-12-24T17:00:00,6,Friuli Venezia Giulia,582,56,638,11464,12102,-173,...,33099,1514,NaN,NaN,46715,898714,332699.0,4.0,NaN,NaN
6,5,2020-12-24T17:00:00,7,Liguria,687,64,751,5207,5958,-219,...,49737,2831,NaN,NaN,58526,689389,316700.0,4.0,I 143027 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-24T17:00:00,8,Emilia-Romagna,2718,201,2919,54380,57299,-726,...,96529,7335,NaN,NaN,161163,2485596,1200124.0,8.0,NaN,NaN
8,7,2020-12-24T17:00:00,9,Toscana,888,161,1049,9920,10969,-388,...,103021,3552,NaN,NaN,117542,1830077,1043441.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-24T17:00:00,10,Umbria,238,38,276,3245,3521,-27,...,23633,591,NaN,NaN,27745,488681,247406.0,4.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-25T17:00:00,1,Piemonte,3029,214,3243,28254,31497,-3373,875,...,7755,NaN,NaN,193375,1648345,972116.0,7.0,NaN,NaN,4341375
1,2020-12-25T17:00:00,2,Valle d'Aosta,84,4,88,340,428,1,11,...,371,NaN,NaN,7130,61109,36873.0,0.0,NaN,NaN,125501
2,2020-12-25T17:00:00,3,Lombardia,3976,522,4498,54212,58710,802,2628,...,24782,NaN,NaN,469883,4765500,2577061.0,20.0,NaN,NaN,10103969
3,2020-12-25T17:00:00,4,Trentino-Alto Adige,453,68,521,11820,12341,1488,336,...,1603,0.0,0.0,49561,794650,311080.0,4.0,0,NaN,1074819
4,2020-12-25T17:00:00,5,Veneto,2582,346,2928,84457,87385,-16637,5010,...,5953,NaN,NaN,234792,3201904,1187079.0,36.0,NaN,NaN,4907704
5,2020-12-25T17:00:00,6,Friuli Venezia Giulia,572,54,626,11458,12084,-18,584,...,1534,NaN,NaN,47299,904549,334240.0,4.0,NaN,NaN,1211357
6,2020-12-25T17:00:00,7,Liguria,663,64,727,4942,5669,-289,445,...,2842,NaN,NaN,58971,693519,317936.0,3.0,I 145443 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-25T17:00:00,8,Emilia-Romagna,2711,206,2917,56063,58980,1681,2127,...,7400,NaN,NaN,163290,2496457,1204670.0,16.0,NaN,NaN,4467118
8,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,...,3558,NaN,NaN,118155,1841106,1047247.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-25T17:00:00,10,Umbria,240,36,276,3270,3546,25,232,...,597,NaN,NaN,27977,490891,248275.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-25T17:00:00,1,Piemonte,3029,214,3243,28254,31497,-3373,875,...,NaN,NaN,193375,1648345,972116.0,7.0,NaN,NaN,4341375,575
1,2020-12-25T17:00:00,2,Valle d'Aosta,84,4,88,340,428,1,11,...,NaN,NaN,7130,61109,36873.0,0.0,NaN,NaN,125501,20
2,2020-12-25T17:00:00,3,Lombardia,3976,522,4498,54212,58710,802,2628,...,NaN,NaN,469883,4765500,2577061.0,20.0,NaN,NaN,10103969,1036
3,2020-12-25T17:00:00,4,Trentino-Alto Adige,453,68,521,11820,12341,1488,336,...,0.0,0.0,49561,794650,311080.0,4.0,0,NaN,1074819,106
4,2020-12-25T17:00:00,5,Veneto,2582,346,2928,84457,87385,-16637,5010,...,NaN,NaN,234792,3201904,1187079.0,36.0,NaN,NaN,4907704,1016
5,2020-12-25T17:00:00,6,Friuli Venezia Giulia,572,54,626,11458,12084,-18,584,...,NaN,NaN,47299,904549,334240.0,4.0,NaN,NaN,1211357,180
6,2020-12-25T17:00:00,7,Liguria,663,64,727,4942,5669,-289,445,...,NaN,NaN,58971,693519,317936.0,3.0,I 145443 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-25T17:00:00,8,Emilia-Romagna,2711,206,2917,56063,58980,1681,2127,...,NaN,NaN,163290,2496457,1204670.0,16.0,NaN,NaN,4467118,634
8,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,...,NaN,NaN,118155,1841106,1047247.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-25T17:00:00,10,Umbria,240,36,276,3270,3546,25,232,...,NaN,NaN,27977,490891,248275.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-25T17:00:00,5,Veneto,2582,346,2928,84457,87385,-16637,5010,...,34.06,-16584,1173,13776,3999.0,94,21553,36.37,0.10208,4.78
1,2020-12-25T17:00:00,3,Lombardia,3976,522,4498,54212,58710,802,2628,...,50.39,1003,-28,27942,9754.0,105,1721,9.41,0.02601,4.65
2,2020-12-25T17:00:00,8,Emilia-Romagna,2711,206,2917,56063,58980,1681,2127,...,32.49,1683,435,10861,4546.0,65,381,19.58,0.04761,3.66
3,2020-12-25T17:00:00,12,Lazio,2666,290,2956,73118,76074,583,1691,...,34.24,610,172,16516,7002.0,31,1077,10.24,0.02883,2.65
4,2020-12-25T17:00:00,16,Puglia,1413,166,1579,52010,53589,458,1011,...,44.99,471,-447,7540,1591.0,26,527,13.41,0.02522,2.14
5,2020-12-25T17:00:00,15,Campania,1449,102,1551,78996,80547,538,1009,...,20.20,577,-147,16355,11689.0,7,464,6.17,0.01744,3.20
6,2020-12-25T17:00:00,1,Piemonte,3029,214,3243,28254,31497,-3373,875,...,37.22,-3259,-182,6080,2223.0,28,4220,14.39,0.02015,4.45
7,2020-12-25T17:00:00,19,Sicilia,995,174,1169,32063,33232,-148,720,...,29.59,-136,-133,6472,4215.0,17,851,11.12,0.01449,1.78
8,2020-12-25T17:00:00,9,Toscana,843,166,1009,10260,11269,300,613,...,31.74,340,50,11029,3806.0,6,307,5.56,0.01647,3.17
9,2020-12-25T17:00:00,11,Marche,398,60,458,9109,9567,152,603,...,41.96,167,188,6711,5062.0,12,439,8.99,0.03971,2.59


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,68,38,106,64.15
1,Lombardia,522,514,1036,50.39
2,Puglia,166,203,369,44.99
3,Marche,60,83,143,41.96
4,Piemonte,214,361,575,37.22
5,Lazio,290,557,847,34.24
6,Veneto,346,670,1016,34.06
7,Emilia-Romagna,206,428,634,32.49
8,Toscana,166,357,523,31.74
9,Umbria,36,81,117,30.77


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2020-12-23T17:00:00,22,P.A. Trento,346,47,393,1273,1666,-337,157,17903,878,NaN,NaN,20447,435028,148970.0,3.0,NaN,"Dei 157 casi odierni, 59 sono molecolari con p..."
608,2020-12-24T17:00:00,21,P.A. Bolzano,157,17,174,8933,9107,-107,142,18729,704,NaN,NaN,28540,352056,160013.0,0.0,NaN,NaN
609,2020-12-24T17:00:00,22,P.A. Trento,326,49,375,1371,1746,80,238,18052,887,NaN,NaN,20685,437593,149319.0,4.0,NaN,"Dei 238 casi odierni, 102 sono molecolari con ..."
610,2020-12-25T17:00:00,21,P.A. Bolzano,147,19,166,10398,10564,1457,182,17447,711,NaN,NaN,28722,354080,161537.0,2.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2020-12-21T17:00:00,4,Trentino-Alto Adige,561,71,632,10787,11419,-1847,102,...,15.0,1717.0,401.0,-1859.0,11.23,5.53,5.94,25.44,2.90,-13.92
302,2020-12-22T17:00:00,4,Trentino-Alto Adige,540,72,612,10746,11358,-61,238,...,19.0,2353.0,652.0,-41.0,11.76,5.39,10.11,36.50,1.41,-0.53
303,2020-12-23T17:00:00,4,Trentino-Alto Adige,516,66,582,10298,10880,-478,288,...,11.0,5587.0,1112.0,-448.0,11.34,5.35,5.15,25.90,-8.33,-4.21
304,2020-12-24T17:00:00,4,Trentino-Alto Adige,483,66,549,10304,10853,-27,380,...,14.0,4451.0,905.0,6.0,12.02,5.06,8.54,41.99,0.00,-0.25


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
